<a href="https://colab.research.google.com/github/ayushiraut01/Resume-Optimization-tool/blob/main/Resume_Optimization_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy pandas spacy sklearn docx2txt pdfminer.six
!python -m spacy download en_core_web_sm

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=59504bb8086d38f454f104263b9a58a42154ceeeeefbfa64fc2e1a6b0794903d
  Stored in directory: /root/.cache/pip/wheels/0f/0e/7a/3094a4ceefe657bff7e12dd9592a9d5b6487ef4338ace0afa6
Successfully built docx2txt


In [4]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.3 MB/s eta 0:00:00


In [5]:
import pdfminer
import docx2txt
import os
from pdfminer.high_level import extract_text

def extract_resume_text(file_path):
    """Extract text from a resume file (PDF or DOCX)."""
    if file_path.endswith('.pdf'):
        text = extract_text(file_path)
    elif file_path.endswith('.docx'):
        text = docx2txt.process(file_path)
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text.strip()


In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")

def analyze_resume(text):
    """Extract relevant details like skills, experience, and education from resume."""
    doc = nlp(text)

    skills = ["Python", "Java", "Machine Learning", "Data Science", "SQL", "Django", "Flask","MYSQL","AWS","laravel"]  # Add more relevant skills
    extracted_skills = [token.text for token in doc if token.text in skills]

    years_experience = sum(1 for token in doc if token.text.isdigit() and int(token.text) <= 50)  # Approximate experience count

    return {
        "skills": extracted_skills,
        "experience": years_experience
    }


In [9]:
!pip install textstat # Install the 'textstat' library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.4 MB/s eta 0:00:00


In [10]:
from textstat import flesch_reading_ease

def score_resume(resume_text, extracted_info):
    """Assign a score to the resume based on various factors."""
    base_score = 50

    # Skills Score
    skill_count = len(extracted_info["skills"])
    skill_score = min(skill_count * 5, 20)  # Max 20 points

    # Experience Score
    experience_score = min(extracted_info["experience"] * 3, 20)  # Max 20 points

    # Readability Score
    readability = flesch_reading_ease(resume_text)
    readability_score = max(0, min((readability / 10), 10))  # Normalize between 0-10

    total_score = base_score + skill_score + experience_score + readability_score
    return round(total_score, 2)


In [11]:
!pip install textstat # Install the 'textstat' library

In [12]:
def evaluate_resume(file_path):
    """Full resume evaluation pipeline."""
    resume_text = extract_resume_text(file_path)
    extracted_info = analyze_resume(resume_text)
    score = score_resume(resume_text, extracted_info)

    print("\nResume Analysis Results:")
    print(f"Extracted Skills: {extracted_info['skills']}")
    print(f"Years of Experience: {extracted_info['experience']}")
    print(f"Resume Score: {score}/100")


In [ ]:
!pip install language_tool_python

In [20]:
import os
import docx2txt
import pdfminer
import spacy
import re
import language_tool_python # This line was missing

from pdfminer.high_level import extract_text
from textstat import flesch_reading_ease
from collections import Counter

# Load NLP Model & Grammar Checker
nlp = spacy.load("en_core_web_sm")
tool = language_tool_python.LanguageToolPublicAPI('en-US')  # Grammar checking

# Predefined Skills List (Expand this as needed)
SKILLS_DB = {"Python", "Java", "Machine Learning", "Data Science", "SQL", "Django", "Flask",
             "NLP", "TensorFlow", "Keras", "Deep Learning", "AWS", "React", "JavaScript",
             "C++", "C#", "HTML", "CSS", "Angular", "Docker", "Kubernetes", "Git"}

# Skill pattern for better extraction
SKILL_PATTERN = re.compile(r'\b(' + '|'.join(SKILLS_DB) + r')\b', re.IGNORECASE)

# ---------------- Step 1: Extract Resume Text ---------------- #
def extract_resume_text(file_path):
    """Extract text from a resume file (PDF or DOCX)."""
    if file_path.endswith('.pdf'):
        text = extract_text(file_path)
    elif file_path.endswith('.docx'):
        text = docx2txt.process(file_path)
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text.strip()

# ---------------- Step 2: Extract Key Information ---------------- #
def analyze_resume(text):
    """Extract skills, experience, and relevant keywords from the resume."""
    doc = nlp(text.lower())  # Convert text to lowercase for uniform processing

    # Extract Skills using Pattern Matching and NLP
    extracted_skills = set(SKILL_PATTERN.findall(text))  # Pattern-based extraction
    extracted_skills.update({token.text for token in doc if token.text in SKILLS_DB})  # NLP-based extraction

    # Approximate Experience Extraction
    experience_years = re.findall(r'(\d+)\s*years?', text)
    years_experience = max(map(int, experience_years), default=0)  # Get max years found

    # Count Keywords Occurrences
    word_freq = Counter([token.text for token in doc if token.is_alpha])

    return {
        "skills": extracted_skills,
        "experience": years_experience,
        "word_frequencies": word_freq
    }

# ---------------- Step 3: Grammar & Formatting Check ---------------- #
def grammar_check(text):
    """Check grammar and spelling issues in the resume."""
    matches = tool.check(text)
    return len(matches), matches[:5]  # Return top 5 grammar issues

def check_bullet_formatting(text):
    """Ensure resume has proper bullet points."""
    bullet_points = re.findall(r'^\s*[-•]', text, re.MULTILINE)
    return len(bullet_points) > 5  # At least 5 bullet points indicate good formatting

# ---------------- Step 4: Job Description Matching ---------------- #
def match_job_description(resume_text, job_description):
    """Compare resume with job description to calculate relevance score."""
    resume_doc = nlp(resume_text.lower())
    job_doc = nlp(job_description.lower())

    resume_words = set([token.text for token in resume_doc if token.is_alpha])
    job_words = set([token.text for token in job_doc if token.is_alpha])

    common_words = resume_words.intersection(job_words)
    missing_skills = job_words - resume_words  # Find missing skills

    relevance_score = (len(common_words) / len(job_words)) * 100
    return round(relevance_score, 2), missing_skills

# ---------------- Step 5: Scoring Algorithm ---------------- #
def score_resume(resume_text, extracted_info, job_description=""):
    """Assign a score based on skills, experience, readability, and job relevance."""
    base_score = 50  # Default base score

    # Skills Score
    skill_count = len(extracted_info["skills"])
    skill_score = min(skill_count * 5, 20)  # Max 20 points

    # Experience Score
    experience_score = min(extracted_info["experience"] * 3, 20)  # Max 20 points

    # Readability Score
    readability = flesch_reading_ease(resume_text)
    readability_score = max(0, min((readability / 10), 10))  # Normalize to 10 points

    # Grammar Check Score
    grammar_issues, _ = grammar_check(resume_text)
    grammar_score = max(0, 10 - grammar_issues)  # Deduct points for grammar errors

    # Bullet Point Score
    bullet_score = 5 if check_bullet_formatting(resume_text) else 0  # 5 points for good formatting

    # Job Relevance Score
    job_match_score = 0
    missing_skills = set()
    if job_description:
        job_match_score, missing_skills = match_job_description(resume_text, job_description)
        job_match_score = min(job_match_score, 20)  # Max 20 points

    total_score = base_score + skill_score + experience_score + readability_score + job_match_score + grammar_score + bullet_score
    return round(total_score, 2), missing_skills

# ---------------- Step 6: Full Evaluation ---------------- #
def evaluate_resume(file_path, job_description=""):
    """Complete Resume Evaluation with Job Matching & Feedback."""
    resume_text = extract_resume_text(file_path)
    extracted_info = analyze_resume(resume_text)
    score, missing_skills = score_resume(resume_text, extracted_info, job_description)

    # Grammar Check
    grammar_issues, grammar_suggestions = grammar_check(resume_text)

    print("\n📄 **Resume Analysis Results:**")
    print(f"✅ Extracted Skills: {', '.join(extracted_info['skills'])}")
    print(f"📆 Years of Experience: {extracted_info['experience']}")
    print(f"📖 Readability Score: {flesch_reading_ease(resume_text):.2f}")

    if job_description:
        print(f"🎯 Job Relevance Score: {match_job_description(resume_text, job_description)[0]}%")
        if missing_skills:
            print(f"🚀 Missing Skills for Job: {', '.join(missing_skills)}")

    print(f"🔍 Grammar Issues Found: {grammar_issues}")
    if grammar_issues > 0:
        print("💡 Top Grammar Suggestions:")
        for issue in grammar_suggestions:
            print(f"  - {issue.message}")

    print(f"📌 Bullet Point Formatting: {'✅ Properly formatted' if check_bullet_formatting(resume_text) else '⚠ Needs better structuring'}")
    print(f"⭐ **Final Resume Score: {score}/100**")

# ---------------- Step 7: Run the Program ---------------- #
if __name__ == "__main__":
    file_path = input("Enter the resume file path: ").strip()
    job_description = input("Enter the job description (or leave blank): ").strip()
    evaluate_resume(file_path, job_description)


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Enter the resume file path: /content/Resume_1 AR.pdf
Enter the job description (or leave blank): ai intern

📄 **Resume Analysis Results:**
✅ Extracted Skills: HTML, CSS, TensorFlow, NLP, machine learning, Python, Flask, SQL, Machine Learning
📆 Years of Experience: 0
📖 Readability Score: -0.10
🎯 Job Relevance Score: 100.0%
🔍 Grammar Issues Found: 25
💡 Top Grammar Suggestions:
  - The official name of this software platform is spelled with a capital “H”.
  - It seems like there are too many consecutive spaces here.
  - It seems like there are too many consecutive spaces here.
  - Possible spelling mistake found.
  - Possible spelling mistake found.
📌 Bullet Point Formatting: ⚠ Needs better structuring
⭐ **Final Resume Score: 90/100**


In [19]:
# Add this at the beginning of your notebook or before using language_tool_python:
!pip install language_tool_python
import language_tool_python

# New Section